In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from google.colab import drive

In [2]:
%cd ..

/


In [ ]:
drive.mount('/home')

In [ ]:
def mesonet_flow(inputs):

  x = layers.Conv2D(8, (3, 3), padding='same', activation = 'relu')(inputs)
  x = layers.BatchNormalization()(x)
  x = layers.MaxPooling2D(pool_size=(2, 2), padding='same')(x)

  x = layers.Conv2D(8, (5, 5), padding='same', activation = 'relu')(x)
  x = layers.BatchNormalization()(x)
  x = layers.MaxPooling2D(pool_size=(2, 2), padding='same')(x)
  
  x = layers.Conv2D(16, (5, 5), padding='same', activation = 'relu')(x)
  x = layers.BatchNormalization()(x)
  x = layers.MaxPooling2D(pool_size=(2, 2), padding='same')(x)

  x = layers.Conv2D(16, (5, 5), padding='same', activation = 'relu')(x)
  x = layers.BatchNormalization()(x)
  x = layers.MaxPooling2D(pool_size=(4, 4), padding='same')(x)

  x = layers.Flatten()(x)
  x = layers.Dropout(0.5)(x)
  x = layers.Dense(16)(x)

  x = layers.LeakyReLU(alpha=0.1)(x)
  x = layers.Dropout(0.5)(x)
  return layers.Dense(1, activation='sigmoid')(x)

In [ ]:
keras.backend.clear_session()
inputs = keras.Input(shape=(100, 100, 3))
outputs = mesonet_flow(inputs)
mesonet = keras.Model(inputs, outputs)

In [ ]:
%cd home/MyDrive/Praca\ Magisterska

In [ ]:
%pwd

In [ ]:
!pip install visualkeras

In [ ]:
%pwd

In [ ]:
import visualkeras

In [ ]:
visualkeras.layered_view(mesonet, legend=True, to_file='output.png').show() 

In [ ]:
import pickle
X_train = pickle.load(open("XColorTrain.pickle", "rb"))
y_train = pickle.load(open("yColorTrain.pickle", "rb"))
X_test = pickle.load(open("XColorTest.pickle", "rb"))
y_test = pickle.load(open("yColorTest.pickle", "rb"))

In [ ]:
mesonet.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

history = mesonet.fit(X_train, y_train, batch_size=32, epochs=10)

In [ ]:
history.history

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
def draw_curves(history, key1='accuracy', ylim1=(0.8, 1.00), 
                key2='loss', ylim2=(0.0, 1.0)):
    plt.figure(figsize=(12,4))
     
    plt.subplot(1, 2, 1)
    plt.plot(history.history[key1], "r--")
    plt.ylabel(key1)
    plt.xlabel('Epoch')
    plt.ylim(ylim1)
 
    plt.subplot(1, 2, 2)
    plt.plot(history.history[key2], "g--")
    plt.ylabel(key2)
    plt.xlabel('Epoch')
    plt.ylim(ylim2)

     
    plt.show()
     
draw_curves(history, key1='accuracy', ylim1=(0.7, 0.1), 
            key2='loss', ylim2=(0.0, 0.8))


In [ ]:
val_loss, val_acc = mesonet.evaluate(X_test, y_test)
print(val_loss, val_acc)

In [ ]:
mesonet.save('mesonet.model')